In [1]:
import zarr
import xarray as xr
from pyproj import CRS, Transformer


# group dir
tmp_zarr_group_dir  = "/playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env/20210123_23z_anl/0_500m_above_ground/CANGLE"

# var dir
tmp_zarr_dir        = "/playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env/20210123_23z_anl/0_500m_above_ground/CANGLE/0_500m_above_ground/CANGLE"

g = zarr.open(tmp_zarr_dir)

# proj y, proj x
g[:].shape

(1059, 1799)

In [2]:
import s3fs
import xarray as xr


# for projecting zarr -> lat/lon
url = "s3://hrrrzarr/sfc/20210601/20210601_00z_anl.zarr"
fs = s3fs.S3FileSystem(anon=True)
chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
chunk_index

<xarray.Dataset> Size: 46MB
Dimensions:     (x: 1799, y: 1059)
Coordinates:
  * x           (x) float64 14kB -2.698e+06 -2.695e+06 ... 2.693e+06 2.696e+06
  * y           (y) float64 8kB -1.587e+06 -1.584e+06 ... 1.584e+06 1.587e+06
Data variables:
    chunk_id    (x, y) object 15MB dask.array<chunksize=(1799, 1059), meta=np.ndarray>
    chunk_x     (x) int32 7kB dask.array<chunksize=(1799,), meta=np.ndarray>
    chunk_y     (y) int32 4kB dask.array<chunksize=(1059,), meta=np.ndarray>
    in_chunk_x  (x) int32 7kB dask.array<chunksize=(1799,), meta=np.ndarray>
    in_chunk_y  (y) int32 4kB dask.array<chunksize=(1059,), meta=np.ndarray>
    index_x     (x) int32 7kB dask.array<chunksize=(1799,), meta=np.ndarray>
    index_y     (y) int32 4kB dask.array<chunksize=(1059,), meta=np.ndarray>
    latitude    (y, x) float64 15MB dask.array<chunksize=(133, 450), meta=np.ndarray>
    longitude   (y, x) float64 15MB dask.array<chunksize=(133, 450), meta=np.ndarray>

In [3]:
chunk_index.longitude.shape

(1059, 1799)

In [3]:
import pandas as pd


df1 = pd.read_csv("/playpen-ssd/levi/ccrfcd-gauge-grids/data/2021-01-01_2025-07-25_gt_p1.csv")
df2 = pd.read_csv("/playpen-ssd/levi/ccrfcd-gauge-grids/data/2021-01-01_2025-07-25_gt_p2.csv")
df  = pd.concat([df1, df2], axis=0).drop("Unnamed: 0", axis=1)
df.head()

,gauge_idx,start_datetime_utc,end_datetime_utc,gauge_acc_in,mrms_q3evap_qpe,lat,lon
0,4564.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.0,0.000000,36.028250,-115.003639
1,4779.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.0,0.000000,36.021861,-114.958750
2,4709.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.0,0.003937,35.925000,-115.117000
3,4724.0,2021-01-23 23:00:00+00:00,2021-01-24 00:00:00+00:00,0.0,0.015748,35.939417,-115.077750
4,4564.0,2021-01-23 23:02:00+00:00,2021-01-24 00:02:00+00:00,0.0,0.000000,36.028250,-115.003639


In [ ]:
import zarr
import timeit
from pathlib import Path
from datetime import datetime, timezone
from glob import glob


VARS_OF_INTEREST = [
    {"name": "DPT", "level": "2m_above_ground", "store": "sfc"},
    {"name": "PWAT", "level": "entire_atmosphere_single_layer", "store": "sfc"},
    {"name": "HGT", "level": "level_of_adiabatic_condensation_from_sfc", "store": "sfc"},
    {"name": "HGT", "level": "highest_tropospheric_freezing_level", "store": "sfc"},
    {"name": "APCP", "level": "surface", "store": "sfc"},
    {"name": "DPT", "level": "925mb", "store": "sfc"},
    {"name": "DPT", "level": "850mb", "store": "sfc"},
    {"name": "DPT", "level": "700mb", "store": "sfc"},
    {"name": "DPT", "level": "500mb", "store": "sfc"},
    {"name": "UGRD", "level": "850mb", "store": "sfc"},
    {"name": "VGRD", "level": "850mb", "store": "sfc"},
    {"name": "UGRD", "level": "700mb", "store": "sfc"},
    {"name": "VGRD", "level": "700mb", "store": "sfc"},
    {"name": "TMP", "level": "2m_above_ground", "store": "sfc"},
    {"name": "PRES", "level": "surface", "store": "sfc"},
    {"name": "HGT", "level": "700mb", "store": "sfc"},
    {"name": "HGT", "level": "850mb", "store": "sfc"},
    {"name": "TMP", "level": "700mb", "store": "sfc"},
    {"name": "TMP", "level": "850mb", "store": "sfc"},
    {"name": "TMP", "level": "500mb", "store": "sfc"},
    {"name": "SPFH", "level": "1000mb", "store": "prs"},
    {"name": "SPFH", "level": "925mb",  "store": "prs"},
    {"name": "SPFH", "level": "850mb",  "store": "prs"},
    {"name": "SPFH", "level": "700mb",  "store": "prs"},
    {"name": "RH",   "level": "925mb",  "store": "prs"},
    {"name": "RH",   "level": "850mb",  "store": "prs"},
    {"name": "RH",   "level": "700mb",  "store": "prs"},
]

HRRR_ENV_DIR = "/playpen-ssd/levi/ccrfcd-gauge-grids/data/hrrr-env"

# TZ assumed UTC
dt_fp_dict = {datetime.strptime(Path(fp).name, "%Y%m%d_%Hz_anl") : fp for fp in glob(f"{HRRR_ENV_DIR}/*")}


def get_hrrr_dir_path(dt: datetime) -> str | None:
    # truncate precision < hour
    y, m, d, h = dt.year, dt.month, dt.day, dt.hour
    truc_dt = datetime(year=y, month=m, day=d, hour=h)
    # return fp if in dict
    if truc_dt in dt_fp_dict: return dt_fp_dict[truc_dt]
    else: return None


def get_vertical_levels(hrrr_dir: str) -> list:
    return [Path(fp).name for fp in glob(f"{hrrr_dir}/*")]


def get_env_vars(hrrr_dir: str) -> dict:
    """
    Given a path to a HRRR dir (i.e., valid date/hr),

    Return
    ---
    ```
        [{
            "var_name": ...,
            "level": ...,
            "zgroup": ...,
            "zarr": ...,
        }]
    ```
    """

    evd = []
    env_var_zarrs = glob(f"{hrrr_dir}/*/*/*/*/.zarray")

    for fp in env_var_zarrs:

        p = Path(fp)

        # path to zarr dir for an atmospheric var
        env_var_zarr_fp  = p.parent
        env_var_group_fp = p.parent.parent.parent
        env_var_name     = p.parent.name
        level_name       = p.parent.parent.name

        for var_of_int in VARS_OF_INTEREST:
            lvl = var_of_int['level']
            nme = var_of_int['name']
            if lvl == level_name and nme == env_var_name:
                evd.append({
                    "level": level_name,
                    "var_name": env_var_name,
                    "var_group": zarr.open_group(env_var_group_fp),
                    "var_zarr": zarr.open_array(env_var_zarr_fp),
                })
                break

    return evd


dir_fp = get_hrrr_dir_path(datetime(year=2021, month=1, day=24, hour=0))
env_var_d = get_env_vars(dir_fp)

`get_hrrr_dir_path`: 0.0000s
`get_env_vars`: 0.0004s


In [5]:
za: zarr.Array = env_var_d[0]["var_zarr"]

In [6]:
import s3fs
import xarray as xr


# for projecting zarr -> lat/lon
url = "s3://hrrrzarr/sfc/20210601/20210601_00z_anl.zarr"
fs = s3fs.S3FileSystem(anon=True)
chunk_index = xr.open_zarr(s3fs.S3Map("s3://hrrrzarr/grid/HRRR_chunk_index.zarr", s3=fs))
chunk_index

<xarray.Dataset> Size: 46MB
Dimensions:     (x: 1799, y: 1059)
Coordinates:
  * x           (x) float64 14kB -2.698e+06 -2.695e+06 ... 2.693e+06 2.696e+06
  * y           (y) float64 8kB -1.587e+06 -1.584e+06 ... 1.584e+06 1.587e+06
Data variables:
    chunk_id    (x, y) object 15MB dask.array<chunksize=(1799, 1059), meta=np.ndarray>
    chunk_x     (x) int32 7kB dask.array<chunksize=(1799,), meta=np.ndarray>
    chunk_y     (y) int32 4kB dask.array<chunksize=(1059,), meta=np.ndarray>
    in_chunk_x  (x) int32 7kB dask.array<chunksize=(1799,), meta=np.ndarray>
    in_chunk_y  (y) int32 4kB dask.array<chunksize=(1059,), meta=np.ndarray>
    index_x     (x) int32 7kB dask.array<chunksize=(1799,), meta=np.ndarray>
    index_y     (y) int32 4kB dask.array<chunksize=(1059,), meta=np.ndarray>
    latitude    (y, x) float64 15MB dask.array<chunksize=(133, 450), meta=np.ndarray>
    longitude   (y, x) float64 15MB dask.array<chunksize=(133, 450), meta=np.ndarray>

In [ ]:
import zarr
import xarray as xr
import contextily as cx
import xyzservices.providers as xyz
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt


from src.plot.magic import MRMS_1H_QPE_PALLETE, MRMS_1H_QPE_BOUNDARIES

_LAT_MIN = 34.751857
_LAT_MAX = 37.103662
_LON_MIN = -116.146925 + 360
_LON_MAX = -113.792819 + 360


# pallete from NSSL MRMS web viewer
colors = MRMS_1H_QPE_PALLETE
boundaries = MRMS_1H_QPE_BOUNDARIES

fig = plt.figure(figsize=(10, 10), dpi=300)
gs  = fig.add_gridspec(nrows=1, ncols=1, wspace=0)

ax1 = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree()) 
ax1.set_extent([_LON_MIN, _LON_MAX, _LAT_MIN, _LAT_MAX], crs=ccrs.PlateCarree())

# add basemap; tell contextily our axis units are degrees
cx.add_basemap(ax1, crs="EPSG:4326", source=xyz.OpenStreetMap.Mapnik, zoom=10)

# ...
arr = env_var_d[0]['var_zarr'][:].clip(0)

# ax1.contourf(arr);
ct1 = ax1.contourf(chunk_index.longitude, chunk_index.latitude, arr, alpha=0.5)

plt.show()

In [7]:
import numpy as np

from scipy.spatial import cKDTree


def build_latlon_index(chunk_index):
    """
    Build a nearest-neighbor index for the HRRR grid.
    Returns (tree, grid_shape, lons_0_360_flag).
    """
    lats = np.asarray(chunk_index.latitude)
    lons = np.asarray(chunk_index.longitude)

    # Normalize lon handling (your notebook uses 0..360)
    lons_0_360 = (np.nanmin(lons) >= 0) and (np.nanmax(lons) <= 360)

    if lons_0_360:
        lons = np.mod(lons, 360.0)  # keep 0..360
    else:
        # keep -180..180 form; still OK, just be consistent
        pass

    pts = np.column_stack([lats.ravel(), lons.ravel()])

    if cKDTree is None:
        raise ImportError("scipy is required for the fast KDTree approach (pip install scipy).")

    tree = cKDTree(pts)
    return tree, lats.shape, lons_0_360


def value_at_latlon(lat, lon, arr, tree, grid_shape, lons_0_360=True):
    """
    Nearest-neighbor value from arr at (lat, lon).
    Returns (value, (iy, ix), distance_degrees).
    """
    
    # if lon in range: 0-360 -> -180-180
    if lons_0_360:
        lon = lon % 360.0

    dist, flat_idx = tree.query([lat, lon], k=1)
    iy, ix = np.unravel_index(flat_idx, grid_shape)
    return float(arr[iy, ix]), (int(iy), int(ix)), float(dist)

In [8]:
# Build index once
tree, grid_shape, lons_0_360 = build_latlon_index(chunk_index)

# Your data
arr = env_var_d[0]["var_zarr"][:].clip(0)

In [9]:
# Example query (note: if you pass -115, the function will wrap it to 245 if grid is 0..360)
val, (iy, ix), dist = value_at_latlon(36.1, -115.2, arr, tree, grid_shape, lons_0_360=lons_0_360)
val, (iy, ix), dist

(93200.0, (491, 372), 0.004096918317703342)

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# for each row, start_datetime
    # hrrr_dir      <-- get hrrr_dir        // find the corresponding HRRR dir for this hour if it exists
    # env_var_zarrs <-- get_zarrs(hrrr_dir) // list of zarrs
    
    # [] // name, val
    # for za in env_var_zarrs
        # lat, lon     <-- row.lat, row.lon
        # val_at_gauge <-- get_value_at(zarr, lat, lon)
        # [].append((za.name, val_at_gauge))


# {datetime: {var: {zarr, location: val@loc}}
zarr_cache = {}
all_rows   = {}


def proc_var(var, dt, lat, lon):

    # unique level, name ID
    comb_name = var["level"] + "_" + var["var_name"]
    
    if comb_name not in zarr_cache[dt]:

        # load zarr into mem; store in cache
        za = var['var_zarr'][:]
        zarr_cache[dt][comb_name] = {
            "zarr": za
        }

    if "val" not in zarr_cache[dt][comb_name]:

        val, (iy, ix), dist = value_at_latlon(lat, lon, za, tree, grid_shape, lons_0_360=lons_0_360)
        zarr_cache[dt][comb_name]["val"] = val

    # load from cache
    val = zarr_cache[dt][comb_name]["val"]

    return (comb_name, val)


def proc_row(i, row):
    
    vals     = row
    dt_str   = " ".join(vals.start_datetime_utc.split(":")[:-3])
    dt       = datetime.strptime(dt_str, "%Y-%m-%d %H")

    # lookup table for caching zarrs
    if dt not in zarr_cache: zarr_cache[dt] = {}
    
    # get path to corresponding hrrr zarr dir
    hrrr_dir = get_hrrr_dir_path(dt)
    lat, lon = vals.lat, vals.lon

    env_vars = get_env_vars(hrrr_dir)
    assert len(env_vars) > 0

    # print("---------------------")
    # print(f"Date: {dt}")

    row_dict = {}

    with ThreadPoolExecutor() as ex:

        procs = [ex.submit(proc_var, var, dt, lat, lon) for var in env_vars]
        for res in as_completed(procs):
            
            result = res.result()
            name, val = result
            row_dict[name] = val

    return i, row_dict


with ThreadPoolExecutor() as ex:

    procs = []
    for i, row in tqdm(enumerate(df.itertuples()), total=len(df)):
        procs.append(ex.submit(proc_row, i, row))

    for proc in tqdm(as_completed(procs), total=len(procs)):
        i, row_dict = proc.result()
        row_dict[i] = row_dict

  0%|          | 0/1557506 [00:00<?, ?it/s]

 36%|███▋      | 566978/1557506 [00:26<00:46, 21245.62it/s]


In [75]:
all_rows

{0: {'surface_PRES': 93460.0,
  'entire_atmosphere_single_layer_PWAT': 9.921875,
  '2m_above_ground_DPT': 269.75,
  '925mb_DPT': 269.75,
  'level_of_adiabatic_condensation_from_sfc_HGT': 1895.0,
  '700mb_VGRD': 5.60546875,
  '700mb_HGT': 2966.0,
  '700mb_UGRD': 3.759765625,
  '700mb_DPT': 262.0,
  '500mb_DPT': 243.25,
  '2m_above_ground_TMP': 285.25,
  '850mb_DPT': 268.25,
  '700mb_TMP': 263.25,
  '500mb_TMP': 247.125,
  'highest_tropospheric_freezing_level_HGT': 1810.0,
  '850mb_TMP': 276.75,
  '850mb_HGT': 1431.0,
  '850mb_UGRD': -0.2275390625,
  '850mb_VGRD': 0.92333984375}}